In [1]:
#Mounting Google Drive from Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#Changing the current working directory to the Google Drive
#%cd /content/drive/My Drive/MLDL2024_project1-Enrico

In [3]:
#!pip install -U fvcore

In [4]:
#Importing the necessary libraries
import os
import torch
import numpy as np
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets.cityscapes import CityscapesCustom
from datasets.gta5 import GTA5Custom
from models.deeplabv2.deeplabv2 import get_deeplab_v2
from models.bisenet.build_bisenet import BiSeNet
from train import train_model
from utils import test_latency_FPS
from utils import test_FLOPs_params

In [5]:
#Set device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Set the manual seeds
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

#Set training parameters
cityscapes_height, cityscapes_width = (64, 128)
gta5_height, gta5_width = (64, 128)
train_batch_size = 4
n_epochs = 10

class_names = ["road", "sidewalk", "building", "wall", "fence", "pole", "traffic_light", "traffic_sign",
                "vegetation", "terrain", "sky", "person", "rider", "car", "truck", "bus", "train",
                "motorcycle", "bicycle"
]

In [6]:
#Create Dataloaders for Cityscapes
cityscapes_dir = os.path.dirname(os.getcwd()) + '/Cityscapes/Cityspaces/'

cityscapes_train_dataset = CityscapesCustom(cityscapes_dir, 'train', cityscapes_height, cityscapes_width)
cityscapes_test_dataset = CityscapesCustom(cityscapes_dir, 'val', cityscapes_height, cityscapes_width)

cityscapes_train_dataloader = DataLoader(cityscapes_train_dataset, train_batch_size, shuffle=True)
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, train_batch_size, shuffle=False)

print(f'Train: {len(cityscapes_train_dataset)} images, divided into {len(cityscapes_train_dataloader)} batches of size {cityscapes_train_dataloader.batch_size}')
print(f'Test: {len(cityscapes_test_dataset)} images, divided into {len(cityscapes_test_dataloader)} batches of size {cityscapes_test_dataloader.batch_size}')

Train: 1572 images, divided into 393 batches of size 4
Test: 500 images, divided into 125 batches of size 4


In [7]:
#Create Dataloader for GTA5
gta5_dir = os.path.dirname(os.getcwd()) + '/GTA5/GTA5/'

gta5_dataset = GTA5Custom(gta5_dir, gta5_height, gta5_width)

gta5_dataloader = DataLoader(gta5_dataset, train_batch_size, shuffle=True)

print(f'GTA5: {len(gta5_dataset)} images, divided into {len(gta5_dataloader)} batches of size {gta5_dataloader.batch_size}')

GTA5: 2500 images, divided into 625 batches of size 4


In [8]:
#Testing Classic semantic segmentation network

#Set up the model with the pretrained weights
DeepLabV2_model = get_deeplab_v2().to(device)

#Set up the loss function and the optimizer
DeepLabV2_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
DeepLabV2_optimizer = optim.Adam(DeepLabV2_model.parameters(), lr=1e-3)

#Test latency and FPS
#print(test_latency_FPS(DeepLabV2_model, device, cityscapes_height, cityscapes_width))

#Test FLOPs and number of parameters
#print(test_FLOPs_params(DeepLabV2_model, device, cityscapes_height, cityscapes_width))

#Train the model
train_model(DeepLabV2_model, DeepLabV2_criterion, DeepLabV2_optimizer,
            cityscapes_train_dataloader, cityscapes_test_dataloader,
            class_names, device, n_epochs, lr_schedule=False, model_name='DeepLabV2')

Deeplab pretraining loading...


Epoch 1/10  29s]
Train mIoU: 7.80%, Test mIoU: 4.29%
road: 41.75% sidewalk: 37.91% building: 0.00% wall: 0.00% fence: 1.93% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 2/10  28s]
Train mIoU: 9.00%, Test mIoU: 6.03%
road: 64.47% sidewalk: 50.13% building: 0.00% wall: 0.00% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 3/10  28s]
Train mIoU: 8.78%, Test mIoU: 6.69%
road: 59.30% sidewalk: 58.79% building: 0.74% wall: 0.00% fence: 8.29% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 4/10  28s]
Train mIoU: 9.03%, Test mIoU: 7.05%
road: 54.07% sidewalk: 41.29% building: 2.07% wall: 0.00% fence: 36.47% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

...

Epoch 6/10  28s]
Train mIoU: 10.69%, Test mIoU: 6.44%
road: 49.88% sidewalk: 43.32% building: 4.37% wall: 0.00% fence: 24.86% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 7/10  27s]
Train mIoU: 11.01%, Test mIoU: 9.41%
road: 60.13% sidewalk: 68.19% building: 2.85% wall: 0.00% fence: 47.71% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 8/10  28s]
Train mIoU: 10.00%, Test mIoU: 10.01%
road: 63.57% sidewalk: 67.37% building: 3.49% wall: 0.00% fence: 55.84% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 9/10  28s]
Train mIoU: 11.53%, Test mIoU: 10.10%
road: 65.15% sidewalk: 69.15% building: 3.13% wall: 0.00% fence: 54.52% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 10/10  28s]
Train mIoU: 12.12%, Test mIoU: 11.80%
road: 72.68% sidewalk: 78.16% building: 0.00% wall: 0.00% fence: 73.41% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 
Best IoU: 11.80% at epoch 10


In [9]:
#Testing Real-Time semantic segmentation network

#Set up the model with the pretrained weights
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

#Set up the loss function and the optimizer for BiSeNet
BiSeNet_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
BiSeNet_optimizer = torch.optim.SGD(BiSeNet_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

#Test latency and FPS
#print(test_latency_FPS(BiSeNet_model, device, cityscapes_height, cityscapes_width))

#Test FLOPs and number of parameters
#print(test_FLOPs_params(BiSeNet_model, device, cityscapes_height, cityscapes_width))

#Train the model
train_model(BiSeNet_model, BiSeNet_criterion, BiSeNet_optimizer,
            cityscapes_train_dataloader, cityscapes_test_dataloader,
            class_names, device, n_epochs, lr_schedule=True, model_name='BiSeNet')

Epoch 1/10  8s]
Train mIoU: 5.90%, Test mIoU: 5.17%
road: 48.30% sidewalk: 49.68% building: 0.00% wall: 0.16% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 2/10  8s]
Train mIoU: 7.97%, Test mIoU: 6.56%
road: 64.36% sidewalk: 59.89% building: 0.00% wall: 0.35% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 3/10  8s]
Train mIoU: 7.38%, Test mIoU: 3.64%
road: 40.75% sidewalk: 26.70% building: 0.00% wall: 0.00% fence: 1.70% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 4/10  8s]
Train mIoU: 7.74%, Test mIoU: 6.33%
road: 52.00% sidewalk: 60.66% building: 0.00% wall: 0.00% fence: 7.56% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 5/10  8s]
Train mIoU: 7.90%, Test mIoU: 6.02%
road: 48.13% sidewalk: 62.14% building: 0.00% wall: 0.00% fence: 4.15% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 6/10  9s]
Train mIoU: 8.07%, Test mIoU: 9.41%
road: 62.52% sidewalk: 66.42% building: 0.00% wall: 0.00% fence: 49.82% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 7/10  9s]
Train mIoU: 9.35%, Test mIoU: 9.68%
road: 66.62% sidewalk: 63.22% building: 0.00% wall: 0.00% fence: 54.14% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 8/10  8s]
Train mIoU: 10.25%, Test mIoU: 10.16%
road: 67.06% sidewalk: 67.61% building: 0.00% wall: 0.00% fence: 58.41% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 9/10  9s]
Train mIoU: 9.97%, Test mIoU: 10.28%
road: 67.11% sidewalk: 68.89% building: 0.00% wall: 0.00% fence: 59.23% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 10/10  9s]
Train mIoU: 10.02%, Test mIoU: 10.28%
road: 67.20% sidewalk: 68.98% building: 0.00% wall: 0.00% fence: 59.20% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 
Best IoU: 10.28% at epoch 10


In [10]:
#Testing Domain Shift problem in semantic segmentation

#Set up the model with the pretrained weights
BiSeNet_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

#Set up the loss function and the optimizer for BiSeNet
BiSeNet_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
BiSeNet_optimizer = torch.optim.SGD(BiSeNet_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

#Train the model
train_model(BiSeNet_model, BiSeNet_criterion, BiSeNet_optimizer,
            gta5_dataloader, cityscapes_test_dataloader,
            class_names, device, n_epochs, lr_schedule=True, model_name='BiSeNet')

Epoch 1/10  9s]
Train mIoU: 3.74%, Test mIoU: 3.18%
road: 55.97% sidewalk: 4.04% building: 0.00% wall: 0.48% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 2/10  9s]
Train mIoU: 6.83%, Test mIoU: 2.18%
road: 41.47% sidewalk: 0.02% building: 0.00% wall: 0.00% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 3/10  8s]
Train mIoU: 8.52%, Test mIoU: 2.89%
road: 54.23% sidewalk: 0.00% building: 0.63% wall: 0.00% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 4/10  9s]
Train mIoU: 9.57%, Test mIoU: 3.21%
road: 60.28% sidewalk: 0.00% building: 0.74% wall: 0.00% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 5/10  9s]
Train mIoU: 10.44%, Test mIoU: 3.13%
road: 58.95% sidewalk: 0.00% building: 0.59% wall: 0.00% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 6/10  8s]
Train mIoU: 9.46%, Test mIoU: 2.90%
road: 54.50% sidewalk: 0.00% building: 0.64% wall: 0.00% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 7/10  9s]
Train mIoU: 10.83%, Test mIoU: 2.82%
road: 52.75% sidewalk: 0.00% building: 0.86% wall: 0.00% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 8/10  8s]
Train mIoU: 9.87%, Test mIoU: 2.64%
road: 49.83% sidewalk: 0.00% building: 0.31% wall: 0.00% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 9/10  8s]
Train mIoU: 9.97%, Test mIoU: 2.72%
road: 51.24% sidewalk: 0.00% building: 0.43% wall: 0.00% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 

Epoch 10/10  8s]
Train mIoU: 10.41%, Test mIoU: 2.75%
road: 51.86% sidewalk: 0.00% building: 0.35% wall: 0.00% fence: 0.00% pole: 0.00% traffic_light: 0.00% traffic_sign: 0.00% vegetation: 0.00% terrain: 0.00% sky: 0.00% person: 0.00% rider: 0.00% car: 0.00% truck: 0.00% bus: 0.00% train: 0.00% motorcycle: 0.00% bicycle: 0.00% 
Best IoU: 3.21% at epoch 4
